# Band structure unfolding
[Link to tutorial](https://www.octopus-code.org/documentation/13/tutorial/periodic_systems/unfolding/)

In this tutorial, we look at how to perform a band-structure unfolding using **Octopus**. This calculation is done in several steps and each one is described below.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import griddata

In [ ]:
!mkdir 4-Band-structure-unfolding

In [ ]:
cd 4-Band-structure-unfolding/

## Supercell ground-state
The first thing to do is to compute the ground state of a supercell. In this example we will use bulk silicon. The input file is similar to the one used in the [Getting started with periodic systems](1-getting-started.ipynb) tutorial, but in the present case we will use a supercell composed of 8 atoms. Here is the corresponding input file:

In [ ]:
%%writefile inp

stdout = 'stdout_gs.txt'
stderr = 'stderr_gs.txt'

CalculationMode = gs

PeriodicDimensions = 3

Spacing = 0.5

a = 10.18
%LatticeParameters
 a | a  | a
90 | 90 |90
%

%ReducedCoordinates
 "Si" | 0.0         | 0.0       | 0.0
 "Si" |   1/2       | 1/2       | 0.0
 "Si" |   1/2       | 0.0       | 1/2
 "Si" |   0.0       | 1/2       | 1/2
 "Si" |   1/4       | 1/4       | 1/4
 "Si" |   1/4 + 1/2 | 1/4 + 1/2 | 1/4
 "Si" |   1/4 + 1/2 | 1/4       | 1/4 + 1/2
 "Si" |   1/4       | 1/4 + 1/2 | 1/4 + 1/2
%

nk = 4
%KPointsGrid
  nk |  nk |  nk
%
KPointsUseSymmetries = yes

All these variables should be familiar from other tutorials. Now run **Octopus** using this input file to obtain the ground-state of the supercell.

In [ ]:
!octopus

## Unfolding setup

After obtaining the ground state of the supercell, we now define the primitive cell on which we want to unfold our supercell and the specific ‘‘k’'-points path that we are interested in. To do this, we take the previous input file and add some new lines:

In [ ]:
%%writefile inp

CalculationMode = gs

PeriodicDimensions = 3

Spacing = 0.5

a = 10.18
%LatticeParameters
 a | a  | a
90 | 90 |90
%

%ReducedCoordinates
 "Si" | 0.0         | 0.0       | 0.0
 "Si" |   1/2       | 1/2       | 0.0
 "Si" |   1/2       | 0.0       | 1/2
 "Si" |   0.0       | 1/2       | 1/2
 "Si" |   1/4       | 1/4       | 1/4
 "Si" |   1/4 + 1/2 | 1/4 + 1/2 | 1/4
 "Si" |   1/4 + 1/2 | 1/4       | 1/4 + 1/2
 "Si" |   1/4       | 1/4 + 1/2 | 1/4 + 1/2
%

nk = 4
%KPointsGrid
  nk |  nk |  nk
%
KPointsUseSymmetries = yes

ExperimentalFeatures = yes
UnfoldMode = unfold_setup
%UnfoldLatticeParameters
  a | a | a
%
%UnfoldLatticeVectors
 0.  | 0.5 | 0.5
 0.5 | 0.  | 0.5
 0.5 | 0.5 | 0.0
%
%UnfoldKPointsPath
 4 | 4 | 8
 0.5 | 0.0 | 0.0 # L point
 0.0 | 0.0 | 0.0 # Gamma point
 0.0 | 0.5 | 0.5 # X point
 1.0 | 1.0 | 1.0 # Another Gamma point
%

Lets see more in detail the input variables that were added:

* [UnfoldMode](https://www.octopus-code.org/documentation//13/variables/utilities/oct-unfold/unfoldmode) = unfold_setup: this variable instructs the utility oct-unfold in which mode we are running. As a first step, we are running in the unfold_setup mode, which generates some files that will be necessary for the next steps.

* [UnfoldLatticeParameters](https://www.octopus-code.org/documentation//13/variables/utilities/oct-unfold/unfoldlatticeparameters) specifies the lattice parameters of the primitive cell. This variable is similar to [LatticeParameters](https://www.octopus-code.org/documentation//13/variables/mesh/simulation_box/latticeparameters).

* [UnfoldLatticeVectors](https://www.octopus-code.org/documentation//13/variables/utilities/oct-unfold/unfoldlatticevectors) specifies the lattice vectors of the primitive cell. This variable is similar to [LatticeVectors](https://www.octopus-code.org/documentation//13/variables/mesh/simulation_box/latticevectors).

* [UnfoldKPointsPath](https://www.octopus-code.org/documentation//13/variables/utilities/oct-unfold/unfoldkpointspath) specifies the ‘‘k’'-points path. The coordinates are indicated as reduced coordinates of the primitive lattice. This variable is similar to [KPointsPath](https://www.octopus-code.org/documentation//13/variables/mesh/kpoints/kpointspath).

Now run the oct-unfold utility. You will obtain two files. The first one (unfold_kpt.dat ) contains the list of ‘‘k’'-points of the specified ‘‘k’'-point path, but expressed in reduced coordinates of the supercell. These are the ‘‘k’'-points for which we need to evaluate the wavefunctions of the supercell. The second file (unfold_gvec.dat ) contains the list of reciprocal lattice vectors that relate the ‘‘k’'-points of the primitive cell to the one in the supercell.

In [ ]:
!oct-unfold

## Obtaining the wavefunctions

In order to perform the unfolding, we need the wavefunctions in the supercell at specific k-points. These points are described in the unfold_kpt.dat file that we obtained in the previous step. To get the wavefunctions, we need to run a non self-consistent calculation. Here is the corresponding input file:

In [ ]:
%%writefile inp

stdout = 'stdout_wf.txt'
stderr = 'stderr_wf.txt'

CalculationMode = unocc

PeriodicDimensions = 3

Spacing = 0.5

a = 10.18
%LatticeParameters
 a | a  | a
90 | 90 |90
%

%ReducedCoordinates
 "Si" | 0.0         | 0.0       | 0.0
 "Si" |   1/2       | 1/2       | 0.0
 "Si" |   1/2       | 0.0       | 1/2
 "Si" |   0.0       | 1/2       | 1/2
 "Si" |   1/4       | 1/4       | 1/4
 "Si" |   1/4 + 1/2 | 1/4 + 1/2 | 1/4
 "Si" |   1/4 + 1/2 | 1/4       | 1/4 + 1/2
 "Si" |   1/4       | 1/4 + 1/2 | 1/4 + 1/2
%

ExperimentalFeatures = yes
UnfoldMode = unfold_setup
%UnfoldLatticeParameters
  a | a | a
%
%UnfoldLatticeVectors
 0.  | 0.5 | 0.5
 0.5 | 0.  | 0.5
 0.5 | 0.5 | 0.0
%
%UnfoldKPointsPath
 4 | 4 | 8
 0.5 | 0.0 | 0.0 # L point
 0.0 | 0.0 | 0.0 # Gamma point
 0.0 | 0.5 | 0.5 # X point
 1.0 | 1.0 | 1.0 # Another Gamma point
%

include unfold_kpt.dat

ExtraStates = 6
ExtraStatesToConverge = 4

In this input file we have changed the calculation mode ([CalculationMode](https://www.octopus-code.org/documentation//13/variables/calculation_modes/calculationmode) = unocc) and replaced all the ‘‘k’'-points related variables (%[KPointsGrid](https://www.octopus-code.org/documentation//13/variables/mesh/kpoints/kpointsgrid), %[KPointsPath](https://www.octopus-code.org/documentation//13/variables/mesh/kpoints/kpointspath), and %[KPoints](https://www.octopus-code.org/documentation//13/variables/mesh/kpoints/kpoints)) by the line:

  include unfold_kpt.dat

When doing a normal band structure calculation one normally also wants to have some extra states, therefore we have used the [ExtraStates](https://www.octopus-code.org/documentation//13/variables/states/extrastates) and [ExtraStatesToConverge](https://www.octopus-code.org/documentation//13/variables/states/extrastatestoconverge) variables, as explained in the [Getting started with periodic systems](1-getting-started.ipynb) tutorial. In this particular case, we are requesting to converge 4 unoccupied states, which all correspond to the first valence band in the folded primitive cell, as the supercell is 4 times larger than the initial cell.

Now run **Octopus** on this input file.

<div style="border: 1px solid black; padding: 10px; background-color: #FFCCCC">
<span style="color: red;"> Warning <br> Note that after you have performed this step, you won’t be able to start a time-dependent calculation from this folder. Indeed, the original ground-state wavefunctions will be replaced by the ones from this last calculation, which are incompatible. Therefore it might be a good idea to create a backup of the restart information before performing this step.</span>
</div>

Alternatively, you can continue in a different directory. Instead of copying the restart information to the new folder, you can use the input variable [RestartOptions](https://www.octopus-code.org/documentation//13/variables/execution/io/restartoptions) to specify explicitely from which directory the restart files are to be read.

In [ ]:
!octopus

## Unfolding

Now that we have computed the states for the required ‘‘k’'-points, we can finally compute the spectral function for each of the ‘‘k’'-points of the specified ‘‘k’'-point path. This is done by changing unfolding mode to be [UnfoldMode](https://www.octopus-code.org/documentation//13/variables/utilities/oct-unfold/unfoldmode) = unfold_run, so the final input file should look like this:

In [ ]:
%%writefile inp 

stdout = 'stdout_unfol.txt'
stderr = 'stderr_unfol.txt'

CalculationMode = unocc

PeriodicDimensions = 3

Spacing = 0.5

a = 10.18
%LatticeParameters
 a | a  | a
90 | 90 |90
%

%ReducedCoordinates
 "Si" | 0.0         | 0.0       | 0.0
 "Si" |   1/2       | 1/2       | 0.0
 "Si" |   1/2       | 0.0       | 1/2
 "Si" |   0.0       | 1/2       | 1/2
 "Si" |   1/4       | 1/4       | 1/4
 "Si" |   1/4 + 1/2 | 1/4 + 1/2 | 1/4
 "Si" |   1/4 + 1/2 | 1/4       | 1/4 + 1/2
 "Si" |   1/4       | 1/4 + 1/2 | 1/4 + 1/2
%

ExperimentalFeatures = yes
UnfoldMode = unfold_run
%UnfoldLatticeParameters
  a | a | a
%
%UnfoldLatticeVectors
 0.  | 0.5 | 0.5
 0.5 | 0.  | 0.5
 0.5 | 0.5 | 0.0
%
%UnfoldKPointsPath
 4 | 4 | 8
 0.5 | 0.0 | 0.0 # L point
 0.0 | 0.0 | 0.0 # Gamma point
 0.0 | 0.5 | 0.5 # X point
 1.0 | 1.0 | 1.0 # Another Gamma point
%

include unfold_kpt.dat

ExtraStates = 6
ExtraStatesToConverge = 4

In [ ]:
!oct-unfold

This will produce the spectral function for the full path (static/ake.dat) and for each individual points of the path (static/ake_XXX.dat). The content of the static/ake.dat file should look like this:

In [ ]:
!cat static/ake.dat

The first column is the coordinate along the ‘‘k’'-point path, the second column is the energy eigenvalue and the last column is the spectral function. There are several ways to plot the information contained in this file. One possibility is to plot it as a heat map. For example, if you are using gnuplot, you can try the following command:

The unfolded bandstructure of silicon is shown below. How does it compare to the bandstructure compute in the [Getting started with periodic systems](1-getting-started.ipynb)?

In [ ]:
data = np.loadtxt("static/ake.dat")

x = data[:, 0]
y = data[:, 1]
intensity = data[:, 2]

x_min, x_max = x.min(), x.max()
y_min, y_max = y.min(), y.max()

num_points = 100

xi = np.linspace(x_min, x_max, num_points)
yi = np.linspace(y_min, y_max, num_points)
xi, yi = np.meshgrid(xi, yi)

zi = griddata((x, y), intensity, (xi, yi), method="nearest")

plt.imshow(zi, extent=[x_min, x_max, y_min, y_max], origin="lower", cmap="viridis")

plt.colorbar(label="Intensity")
plt.show()

Unfolded band structure of bulk silicon.

Note that it is possible to change the energy range and the energy resolution for the unfolded band structure. This is done by specifying the variables [UnfoldMinEnergy](https://www.octopus-code.org/documentation//13/variables/utilities/oct-unfold/unfoldminenergy), [UnfoldMaxEnergy](https://www.octopus-code.org/documentation//13/variables/utilities/oct-unfold/unfoldmaxenergy), and [UnfoldEnergyStep](https://www.octopus-code.org/documentation//13/variables/utilities/oct-unfold/unfoldenergystep). It is also important to note here that the code needs to read all the unoccupied wavefunctions and therefore might need a large amount of memory, so make sure enough memory is available.

[Go to *5-Sternheimer.ipynb*](5-Sternheimer.ipynb)